In [1]:
import os
import torch

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from segment.data.aapm_thoracic.aapm import AAPM
from segment.data.aapm_thoracic.aapm import Patient

In [2]:
PATH = '/home/ygx/data/aapm17/labeled'

In [3]:
aapm = AAPM(PATH)

In [4]:
loader = DataLoader(aapm, 1)

In [5]:
for idx, (img, mask) in enumerate(loader):
    print(f'Image: {img.shape}, Mask: {mask.shape}')

Image: torch.Size([1, 512, 512, 122]), Mask: torch.Size([1, 512, 512, 122])
Image: torch.Size([1, 512, 512, 134]), Mask: torch.Size([1, 512, 512, 134])
Image: torch.Size([1, 512, 512, 140]), Mask: torch.Size([1, 512, 512, 140])
Image: torch.Size([1, 512, 512, 152]), Mask: torch.Size([1, 512, 512, 152])
Image: torch.Size([1, 512, 512, 144]), Mask: torch.Size([1, 512, 512, 144])
Image: torch.Size([1, 512, 512, 148]), Mask: torch.Size([1, 512, 512, 148])
Image: torch.Size([1, 512, 512, 136]), Mask: torch.Size([1, 512, 512, 136])
Image: torch.Size([1, 512, 512, 161]), Mask: torch.Size([1, 512, 512, 161])
Image: torch.Size([1, 512, 512, 168]), Mask: torch.Size([1, 512, 512, 168])
Image: torch.Size([1, 512, 512, 160]), Mask: torch.Size([1, 512, 512, 160])
Image: torch.Size([1, 512, 512, 133]), Mask: torch.Size([1, 512, 512, 133])
Image: torch.Size([1, 512, 512, 131]), Mask: torch.Size([1, 512, 512, 131])
Image: torch.Size([1, 512, 512, 160]), Mask: torch.Size([1, 512, 512, 160])
Image: torch